## Checking out the other data sources & Game Plan
* cd rt_segment_speeds && pip install altair_transform && pip install -r requirements.txt && cd ../_shared_utils && make setup_env

In [1]:
import _report_utils
import altair as alt
import calitp_data_analysis.magics
import geopandas as gpd
import great_tables as gt
import pandas as pd
from calitp_data_analysis import calitp_color_palette as cp
from great_tables import md
from IPython.display import HTML, Markdown, display
from segment_speed_utils.project_vars import RT_SCHED_GCS
from shared_utils import catalog_utils, rt_dates, rt_utils

alt.renderers.enable("html")
alt.data_transformers.enable("default", max_rows=None)
from typing import List, Union

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
name = "SBMTD Schedule"

In [4]:
date = "2024-03-13"

In [5]:
year = "2024"

In [6]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

## monthly_scheduled_service
* https://dbt-docs.calitp.org/#!/model/model.calitp_warehouse.fct_monthly_route_service_by_timeofday
* Total scheduled service hours that occurred for the route for this month, day_type, and time_of_day.

In [7]:
# monthly_schd_service_url = f"{GTFS_DATA_DICT.schedule_tables.gcs_dir}{GTFS_DATA_DICT.schedule_tables.monthly_scheduled_service}_{year}.parquet"

In [8]:
# monthly_scheduled_service_df = pd.read_parquet(monthly_schd_service_url)

In [9]:
monthly_scheduled_service_df_agg = _report_utils.summarize_monthly(year, name)

In [10]:
_report_utils.single_bar_chart_dropdown(
    monthly_scheduled_service_df_agg,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Average Total Service Hours across all Routes per Month in 2023",
    "full_date",
)

/opt/conda/lib/python3.9/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)

## segment_speeds
* For mapping

In [ ]:
seg_speeds_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_segment_speeds}.parquet"

In [ ]:
seg_speeds_gdf = gpd.read_parquet(seg_speeds_url)

### digest/operator_profiles

In [ ]:
seg_speeds_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"